In [2]:
# import cvxpy as cp
import numpy as np
from numdifftools import Hessian, Gradient, Jacobian
from alpha_algos import get_alpha
import quadprog
from numpy import linalg as la


In [3]:

def quad_solver(Q, c, A, eq_constraints, G, ineq_constraints):
    d = cp.Variable(len(c))
    prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(d, Q) + c.T @ d),
                 [G @ d <= ineq_constraints,
                  A @ d == eq_constraints])
    prob.solve()
    return d.value

In [4]:
def nearestPD(A):
    B = (A + A.T) / 2
    _, s, V = la.svd(B)
    H = np.dot(V.T, np.dot(np.diag(s), V))
    A2 = (B + H) / 2
    A3 = (A2 + A2.T) / 2
    if isPD(A3):
        return A3
    spacing = np.spacing(la.norm(A))
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1
    return A3

def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False

In [5]:
def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T).astype("double")   # make sure P is symmetric
    if not isPD(qp_G):
        qp_G = nearestPD(qp_G)
        print("WARNING: Q NOT PD")
    qp_a = -q.astype("double")
    if A is not None:
        qp_C = -np.vstack([A, G]).T.astype("double")
        qp_b = -np.hstack([b, h]).astype("double")
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = -G.T.astype("double")
        qp_b = -h.astype("double")
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

In [6]:
def SQP(f, g, h, x_k0, VERBOSE = True):
    f_gradient = Gradient(f)
    f_hessian = Hessian(f)
    h_jacobian = Jacobian(h)
    g_jacobian = Jacobian(g)
    n_iter = 0
    while n_iter < MAX_ITER:
        n_iter +=1
        c = f_gradient(x_k0)
        Q = f_hessian(x_k0)
        A = h_jacobian(x_k0)
        G = g_jacobian(x_k0)
        ineq_constraints = -g(x_k0).flatten()
        eq_constraints = -h(x_k0).flatten()
        d = quadprog_solve_qp(Q, c, G, ineq_constraints, A, eq_constraints)
        
        #quad_solver(Q, c, A, eq_constraints, G, ineq_constraints)
        alpha = 1#get_alpha("exact", x_k0, f, f_gradient, d)
        x_k_new = x_k0 + d
        norm_change = np.linalg.norm(x_k_new - x_k0)
        x_k0 = x_k_new
        if VERBOSE:
            print(f"\n----- ITERATION {n_iter}: -------")
            print(f"Gradient f: {c.round(4)}")
            print(f"Hessian f: {Q.round(4)}")
            print(f"Jacobian h: {A.round(4)}")
            print(f"Jacobian g: {G.round(4)}")
            print(f"g(x): {-ineq_constraints.round(4)}")
            print(f"h(x): {eq_constraints.round(4)}")
            print(f"Direction: {d.round(4)}")
            print(f"Alpha: {round(alpha, 4)}")
            print(f"New x: {x_k_new.round(4)}")
            print(f"||Xk+1 - xk||: {norm_change}")
        if norm_change < EPSILON:
            break
    return x_k_new

In [7]:
f = lambda x: (x[1]-x[0]**2)**2 + (1-x[0])**2
h = lambda x: np.array([0])
g = lambda x: 0.25 * x[0]**2 + 2 - x[1]

EPSILON = 10e-6
MAX_ITER = 1000

x_0 = np.array([2,2])
SQP(f, g, h, x_0)


----- ITERATION 1: -------
Gradient f: [18. -4.]
Hessian f: [[42. -8.]
 [-8.  2.]]
Jacobian h: [[0. 0.]]
Jacobian g: [[ 1. -1.]]
g(x): [1.]
h(x): [0]
Direction: [-0.2  1.2]
Alpha: 1
New x: [1.8 3.2]
||Xk+1 - xk||: 1.2165525060596272

----- ITERATION 2: -------
Gradient f: [ 1.888 -0.08 ]
Hessian f: [[28.08 -7.2 ]
 [-7.2   2.  ]]
Jacobian h: [[0. 0.]]
Jacobian g: [[ 0.9 -1. ]]
g(x): [-0.39]
h(x): [0]
Direction: [-0.2343 -0.6009]
Alpha: 1
New x: [1.5657 2.5991]
||Xk+1 - xk||: 0.6449220057164247

----- ITERATION 3: -------
Gradient f: [0.2065 0.2954]
Hessian f: [[21.0208 -6.2628]
 [-6.2628  2.    ]]
Jacobian h: [[0. 0.]]
Jacobian g: [[ 0.7829 -1.    ]]
g(x): [0.0137]
h(x): [0]
Direction: [-0.03   -0.0098]
Alpha: 1
New x: [1.5357 2.5894]
||Xk+1 - xk||: 0.0315509335812169

----- ITERATION 4: -------
Gradient f: [-0.3474  0.4619]
Hessian f: [[19.9433 -6.1428]
 [-6.1428  2.    ]]
Jacobian h: [[0. 0.]]
Jacobian g: [[ 0.7679 -1.    ]]
g(x): [0.0002]
h(x): [0]
Direction: [-0.0005 -0.0002]
Alpha

array([1.53518375, 2.58919729])

In [8]:
# Minimize f(x) S.t h(x) = 0 and g(x) < 0
f = lambda x: 6*x[0]/x[1] + x[1]/x[0]**2
h = lambda x: x[0]*x[1]-2
g = lambda x: 1-x[0]-x[1]

x_0 = np.array([2,1])
SQP(f, g, h, x_0)


----- ITERATION 1: -------
Gradient f: [  5.75 -11.75]
Hessian f: [[ 0.375 -6.25 ]
 [-6.25  24.   ]]
Jacobian h: [[1. 2.]]
Jacobian g: [[-1. -1.]]
g(x): [-2]
h(x): [0]
Direction: [-0.0795  0.0398]
Alpha: 1
New x: [1.9205 1.0398]
||Xk+1 - xk||: 0.08891776882380728

----- ITERATION 2: -------
Gradient f: [  5.4769 -10.3872]
Hessian f: [[ 0.4586 -5.8322]
 [-5.8322 20.5014]]
Jacobian h: [[1.0398 1.9205]]
Jacobian g: [[-1. -1.]]
g(x): [-1.9602]
h(x): [0.0032]
Direction: [-0.8886  0.4827]
Alpha: 1
New x: [1.0319 1.5225]
||Xk+1 - xk||: 1.0112337625581265

----- ITERATION 3: -------
Gradient f: [ 1.1696 -1.7319]
Hessian f: [[ 8.0568 -4.4087]
 [-4.4087  3.5087]]
Jacobian h: [[1.5225 1.0319]]
Jacobian g: [[-1. -1.]]
g(x): [-1.5544]
h(x): [0.4289]
Direction: [0.009  0.4023]
Alpha: 1
New x: [1.0409 1.9248]
||Xk+1 - xk||: 0.4024381854996722

----- ITERATION 4: -------
Gradient f: [-0.2959 -0.7629]
Hessian f: [[ 9.8364 -3.3926]
 [-3.3926  1.7516]]
Jacobian h: [[1.9248 1.0409]]
Jacobian g: [[-1. -1.

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


array([1.00000002, 1.99999995])

In [27]:
def is_pd(x):
    return np.all(np.linalg.eigvals(x) > 0)

def is_psd(x):
    return np.all(np.linalg.eigvals(x) >= 0)

In [28]:
f = lambda x: 2*x[0]**3 + 3*x[1]**2+3*x[0]**2*x[1]-24*x[1]

In [29]:
H, G = Hessian(f), Gradient(f)

In [63]:
x = np.array([4, -4])


In [64]:
G(x)

array([2.21296679e-14, 1.71517833e-14])

In [65]:
H(x)

array([[24., 24.],
       [24.,  6.]])

In [43]:
is_pd(H(x))

True

In [44]:
is_psd(H(x))

True